## Obtaining financial data for Amazon

TO DO:

1. Restore the list of indicators and the query to select EPS
2. Find underlying EPS figures -- earnings after dividends and number of shares outstanding.
3. Look at current trends around PE
4. If PE hits 20, how much will earnings be?
5. Possible to find the current short interest?


In [1]:
import requests
import config
import pandas as pd
from StringIO import StringIO

#### Company list

In [2]:
base = 'https://api.usfundamentals.com/v1/'
param = 'companies/xbrl?&format=json&token={}'.format(config.us_fun_key)

In [3]:
r = requests.get('{}{}'.format(base, param)).json()

In [4]:
companies = pd.DataFrame(r).set_index('name_latest')['company_id'].to_dict()

#### List of Amazon Indicators

In [73]:
#indicators = pd.read_csv(StringIO(r.text))['indicator_id'].unique()

In [74]:
#indicators

array(['EarningsPerShareDiluted'], dtype=object)

#### Earnings per share

In [2]:
base = 'https://api.usfundamentals.com/v1/indicators/xbrl?'
param = 'companies=1018724&frequency=q&token={}'.format(config.us_fun_key)
#ind = 'indicators=EarningsPerShareDiluted&'
#param = '{}companies=1018724&frequency=q&token={}'.format(ind, config.us_fun_key)

In [3]:
r = requests.get('{}{}'.format(base, param))

In [8]:
indicators = pd.read_csv(StringIO(r.text))['indicator_id'].unique()
indicators

array(['AccountsPayableCurrent', 'AccountsReceivableNetCurrent',
       'AccruedLiabilitiesCurrent',
       'AccruedLiabilitiesForUnredeeemedGiftCards',
       'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment',
       'AccumulatedOtherComprehensiveIncomeLossAvailableForSaleSecuritiesAdjustmentNetOfTax',
       'AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax',
       'AccumulatedOtherComprehensiveIncomeLossNetOfTax',
       'AdditionalPaidInCapital', 'AllocatedShareBasedCompensationExpense',
       'AllowanceForDoubtfulAccountsReceivable',
       'AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount',
       'Assets', 'AssetsCurrent',
       'BusinessAcquisitionCostOfAcquiredEntityCashPaid',
       'BusinessAcquisitionCostOfAcquiredEntityEquityInterestsIssuedAndIssuable',
       'BusinessAcquisitionCostOfAcquiredEntityOtherNoncashConsideration',
       'BusinessAcquisitionCostOfAcquiredEntityPurchasePrice',
 

In [26]:
EPS = pd.read_csv(StringIO(r.text)).query('indicator_id == "EarningsPerShareDiluted"').T[2:]
#id2 = 'WeightedAverageNumberOfDilutedSharesOutstanding'
SHARES = pd.read_csv(StringIO(r.text)).query('indicator_id == "WeightedAverageNumberOfDilutedSharesOutstanding"').T[2:]
#id3 = 'NetIncomeLoss'
NI = pd.read_csv(StringIO(r.text)).query('indicator_id == "NetIncomeLoss"').T[2:]

In [33]:
#pd.join([EPS, SHARES, NI])
#EPS.merge(SHARES).merge(NI)
EPS.reset_index().merge(NI.reset_index()).merge(SHARES.reset_index())
#for s in [EPS, SHARES, NI]:
#    s['date'] = s.index.map(lambda x: pd.Period(x).end_time.date())
#    s = s.set_index('date').rolling(4).sum()

,index,99,171,259
0,2011Q1,0.44,2.01e+08,4.59e+08
1,2011Q2,0.41,1.91e+08,4.6e+08
2,2011Q3,0.14,6.3e+07,4.61e+08
3,2011Q4,0.38,1.76e+08,-9.19e+08
4,2012Q1,0.28,1.3e+08,4.6e+08
5,2012Q2,0.01,7e+06,4.58e+08
6,2012Q3,-0.6,-2.74e+08,4.6e+08
7,2012Q4,0.22,9.8e+07,-9.25e+08
8,2013Q1,0.18,8.2e+07,4.63e+08
9,2013Q2,-0.02,-7e+06,4.56e+08


In [13]:
NI

,171,date
2011Q1,2.01e+08,2011-03-31
2011Q2,1.91e+08,2011-06-30
2011Q3,6.3e+07,2011-09-30
2011Q4,1.76e+08,2011-12-31
2012Q1,1.3e+08,2012-03-31
2012Q2,7e+06,2012-06-30
2012Q3,-2.74e+08,2012-09-30
2012Q4,9.8e+07,2012-12-31
2013Q1,8.2e+07,2013-03-31
2013Q2,-7e+06,2013-06-30


### Stock price history

In [10]:
base = 'https://www.quandl.com/api/v3/datatables/WIKI/PRICES.json?'
param = 'ticker=AMZN&qopts.columns=date,close&api_key={}'.format(config.quandl_key)

In [11]:
r = requests.get('{}{}'.format(base, param)).json()

In [46]:
df = pd.DataFrame(r['datatable']['data']).set_index(0)

In [49]:
idx = pd.date_range(df.index[0], df.index[-1])
df.index = pd.DatetimeIndex(df.index)
df = df.reindex(idx, fill_value=None).fillna(method='ffill')

In [70]:
PE = EPS.merge(df, left_index=True, right_index=True, how='left')

In [71]:
PE['PE_ratio'] = PE[1] / PE[0]

In [72]:
PE

,0,1,PE_ratio
date,,,
2011-03-31,NaN,180.130,NaN
2011-06-30,NaN,204.490,NaN
2011-09-30,NaN,216.230,NaN
2011-12-31,1.37,173.100,126.350365
2012-03-31,1.21,202.510,167.363636
2012-06-30,0.81,228.350,281.913580
2012-09-30,0.07,254.320,3633.142857
2012-12-31,-0.09,250.870,-2787.444444
2013-03-31,-0.19,266.490,-1402.578947
